## Gathering PL

In [6]:
import os

In [52]:
tsv_file = '/private/home/qiantong/w2v/data/librivox/train_36s_real.tsv'
# root = '/checkpoint/qiantong/w2v/pl/960h_rescore_lv/{}_hypo.word-checkpoint_best.pt-train_36s_real.txt'
# n_workers = 400

# tsv_file = '/checkpoint/abaevski/data/speech/libri/960h/wav2vec/raw/train.tsv'
# root = '/checkpoint/qiantong/w2v/pl/ls_10m_rescore/{}_hypo.word-wav2vec_big_10m.pt-train.txt'
# root = '/checkpoint/qiantong/w2v/pl/ls_1h_rescore/{}_hypo.word-wav2vec_big_1h.pt-train.txt'
# root = '/checkpoint/qiantong/w2v/pl/ls_10h_rescore/{}_hypo.word-wav2vec_big_10h.pt-train.txt'
# root = '/checkpoint/qiantong/w2v/pl/ls_100h_rescore/{}_hypo.word-wav2vec_big_100h.pt-train.txt'
# n_workers = 50

root = '/checkpoint/qiantong/w2v/pl/1h_rescore_lv/{}_hypo.word-wav2vec_vox_new_1h.pt-train_36s_real.txt'
n_workers = 300

In [53]:
trans = {}
for i in range(n_workers):
    if not os.path.exists(root.format(i)):
        continue
    with open(root.format(i)) as f:
        for line in f:
            line = line.strip()
            try:
                if line.startswith('(None-'):
                    sp = line[:-1].split('-')
                    trans[int(sp[1])] = ''
                else:
                    sp = line[:-1].split(' (None-')
                    trans[int(sp[1])] = sp[0]
            except:
                print(line)
                print(sp)
               
print(len(trans))

12283659


In [54]:
print(len(trans))
sorted_items = sorted(trans.items()) # 15863907

12283659


In [19]:
def w2l(word):
    return " ".join(list(word) + ['|'])

min_duration = 16000 # 1s
max_duration = 576000 # 36s

In [55]:
out_name = root.split('/')[-2]
out_dir = "/private/home/qiantong/w2v/data/" + out_name + '/'

with open(tsv_file) as ftsv:
    lines = ftsv.readlines()
    with open(out_dir + 'train.tsv', 'w') as fout, open(out_dir + 'train.ltr', 'w') as fltr:
        fout.write(lines[0])
        for i, line in sorted_items:
            line = line.strip()
            if len(line) == 0:
                continue
                
            duration = int(lines[i + 1].split('\t')[-1])
            if duration < min_duration:
                continue
            if duration > max_duration:
                continue

            out_line = ""
            for word in line.upper().split():
                out_line += " " + w2l(word)
            fltr.write(out_line[1:] + '\n')
            fout.write(lines[i + 1])

In [56]:
with open(out_dir + 'train.ltr') as fltr:
    with open(out_dir + 'train.tsv') as ftsv:
        with open('/private/home/qiantong/push_numbers/lists/w2v_pl_{}.lst'.format(out_name), 'w') as fout:
            lines = ftsv.readlines()
            root = lines[0].strip() + '/'
            for i, line in enumerate(fltr):
                meta = lines[i + 1].strip()
                duration = int(meta.split('\t')[-1]) / 16
                meta = meta.split('\t')[0]
                trans = ' '.join(''.join(line.split(' ')).split('|')).lower()
                #print('\t'.join([str(i), root + meta, str(duration), trans]))
                #break
                fout.write('\t'.join([str(i), root + meta, str(duration), trans]))

In [38]:
groud_truth = {}

with open('/private/home/qiantong/push_numbers/lists/train-all.lst') as f:
    for line in f:
        sp = line.strip().split()
        key = '/'.join(sp[1].split('/')[-4:])
        groud_truth[key] = sp[3:]

In [51]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

total_dist = 0.
total_length = 0.
with open('/private/home/qiantong/push_numbers/lists/w2v_pl_{}.lst'.format(out_name)) as f:
    for line in f:
        sp = line.strip().split()
        key = '/'.join(sp[1].split('/')[-4:])
        trans = sp[3:]
        targ = groud_truth[key]
        total_length += len(targ)
        dist = levenshteinDistance(trans, targ)
        total_dist += dist
print(total_dist / total_length)
    

0.02059062362817795


In [59]:
with open('/private/home/qiantong/push_numbers/lists/librivox.cut.sub36s.no_overlap_lm.lst') as f:
    duration = 0
    for line in f:
        sp = line.split()
        duration += float(sp[2]) / 1000
#         print(sp)
#         break
        
    print(duration/3600)

55802.783323872114


In [4]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [1]:
truth = {}

fileroot = "/private/home/qiantong/push_numbers/lists/"
for ff in ['train-clean-360.lst', 'train-other-500.lst']:
    with open(fileroot + ff) as f:
        for line in f:
            sp = line.split()
            truth[sp[0]] = sp[3:]

In [2]:
for k, v in truth.items():
    print(k, v)
    break

train-clean-360-100-121669-0000 ['tom', 'the', "piper's", 'son']


In [6]:
fileroot = "/checkpoint/qiantong/wav2letter_experiments/viterbi_ipl/{}/generated_pl/epoch_140/{}.lst"

for run in ['1', '1_runnamenov16', '1_runnamenov18']:
    print(' --- {} ---'.format(run))
    error = 0.
    nword = 0.

    for i in range(64):
        with open(fileroot.format(run, i)) as f:
            for line in f:
                sp = line.split('\t')
                tar = truth[sp[0]]
                nword += len(tar)
                error += levenshteinDistance(tar, sp[-1].split())

    print(error / nword)

 --- 1 ---
0.20721683258753343
 --- 1_runnamenov16 ---
0.21097352921457782
 --- 1_runnamenov18 ---
0.21250640271797383


In [20]:
lists = "/checkpoint/jacobkahn/data/lists/librispeech/train-clean-100.lst,/checkpoint/jacobkahn/data/lists/librispeech/train-clean-360.lst,/checkpoint/jacobkahn/data/lists/librispeech/train-other-500.lst,/checkpoint/wav2letter/data/swbd_lists/noNL/switchboard.16khz.lst,/checkpoint/wav2letter/data/swbd_lists/noNL/fisher.16khz.lst,/checkpoint/wav2letter/data/commonvoice/lists/train.lst,/checkpoint/qiantong/wsj/si284.lst,/checkpoint/wav2letter/data/tedlium/lists/train.lst"

with open('/private/home/qiantong/wav2letter_experiments/256_GPU/join_train_filtered.lst', 'w') as fout:
    for ff in lists.split(','):
        with open(ff) as f:
            for line in f:
                line = line.strip()
                sp = line.split('\t')
                if len(sp) == 1:
                    tmp = line.split()
                    if len(tmp) < 4:
                        continue
                    sp = tmp[:3]
                    sp.append(' '.join(tmp[3:]))
                    
                if len(sp) != 4:
                    continue
                if float(sp[-2]) > 35000:
                    continue
                if len(sp[-1]) > 500:
#                     print(sp)
                    continue
                
                fout.write('\t'.join(sp) + '\n')

In [7]:
def levenshteinDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [44]:
offset = 5
interval = 5

decoder_scores = []
lm_scores = []
am_scores = []
wer = []
lens = []

with open('/private/home/abaevski/tmp/qiantong_lexfree.txt') as f:
    lines = f.readlines()
    n = len(lines)
#     n = 50005
    for i in range(offset, n, interval):
        index = lines[i].split(':')[-2][-1]
        if index == '0':
            decoder_scores .append([])
            lm_scores .append([])
            am_scores .append([])
            wer .append([])
            lens.append([])
        
        hypo = lines[i].split(':')[-1].split()
        target = lines[i + 1].split(':')[-1].split()
        decoder_score = float(lines[i + 2].split(':')[-1])
        lm_score = float(lines[i + 2].split(',')[-2].split()[-1])
        am_score = float(lines[i + 2].split(',')[-3].split()[-1])
#         print(decoder_score, lm_score)
#         print(len(hypo), len(target))
#         break
        wer[-1].append(levenshteinDistance(hypo, target) / len(target))
        decoder_scores[-1].append(decoder_score)
        lm_scores[-1].append(lm_score)
        am_scores[-1].append(am_score)
        lens[-1].append(len(target))
        


ValueError: could not convert string to float: '15:02:32'

In [50]:
total_decoder_err = 0.
total_lm_err = 0.
total_len = 0.

for i in range(len(wer)):
    total_decoder_err += lens[i][0] * wer[i][0]
    total_len += lens[i][0]
    
    max_lmscore = -1010000
    lm_err = 0.
    for j in range(len(wer[i])):
        if max_lmscore < am_scores[i][j]:
            max_lmscore = am_scores[i][j]
            lm_err = lens[i][j] * wer[i][j]
    total_lm_err += lm_err
    
print(total_decoder_err / total_len)
print(total_lm_err / total_len)

0.1326918368867553
0.1318935144283479


In [49]:
for lmweight in range(-10, 10):
    lmweight = lmweight / 10
    total_err = 0.
    total_len = 0.

    for i in range(len(wer)):
        total_len += lens[i][0]
        
        max_lmscore = -1010000
        err = 0.
        for j in range(len(wer[i])):
            combined_score = am_scores[i][j] + lmweight*lm_scores[i][j]
            if max_lmscore < combined_score:
                max_lmscore = combined_score
                err = lens[i][j] * wer[i][j]
#                 print(j)
        total_err += err
    
    print(lmweight, total_err / total_len)

-1.0 0.13375094468157578
-0.9 0.1335859580401716
-0.8 0.13350612579433085
-0.7 0.1333890385004311
-0.6 0.13321872970930418
-0.5 0.1330643873673454
-0.4 0.13294730007344566
-0.3 0.13260136034146913
-0.2 0.13217558836365184
-0.1 0.1318988365780706
0.0 0.1318935144283479
0.1 0.13360724663906245
0.2 0.13501229416585947
0.3 0.13599689186456193
0.4 0.13662490553184242
0.5 0.13702406676104611
0.6 0.13723163060023205
0.7 0.13747644948747698
0.8 0.1376148253802676
0.9 0.13767336902721747
